### 기본 세팅

In [1]:
import time
from datetime import date, timedelta
import urllib
import pandas as pd

from selenium import webdriver
from bs4 import BeautifulSoup 

In [2]:
# 어제 날짜 저장
def get_yesterday():
    today     = date.today()             # 오늘 날짜
    yesterday = today - timedelta(1)     # 어제 날짜

    year = str(yesterday.year)
    mon  = str(yesterday.month)
    if int(mon) < 10:
        mon = '0'+str(mon)
    day  = str(yesterday.day)
    if int(day) < 10:
        day = '0'+str(day)

    return year+'.'+mon+'.'+day

### 디시인사이드 주식갤러리 크롤링

In [3]:
def get_dcinside_info(keyword):
    
    # 크롤링 정보(크롤링 날짜, 키워드 인코딩)
    sleeptime = 0.5     # 프로세스 일시정지 시간
    yesterday = get_yesterday()   
    day_back = yesterday[5:]
    
    encoded_KEYWORD = urllib.parse.quote(keyword)
    num = 1


    
    title_ = []     # 제목 리스트
    url_   = []     # 주소 리스트
    time_  = []     # 게시날짜 리스트
    count_ = []     # 조회수 리스트
    
    # 최대 30페이지까지 크롤링
    while num <= 30:
        # 드라이버 실행
        driver = webdriver.Chrome('chromedriver.exe')
        time.sleep(sleeptime)
       
    
        
        # 디시인사이드 접속
        new_url = 'https://gall.dcinside.com/board/lists/?id=neostock&page='+str(num)+'&search_pos=&s_type=search_subject_memo&s_keyword='+encoded_KEYWORD
        driver.get(new_url)
        time.sleep(sleeptime)


        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        links = soup.find_all(class_ = 'gall_tit')       # 게시글 객체 파싱(제목, 연결 링크 포함)
        count = soup.find_all(class_ = 'gall_count')     # 조회수 파싱 
        date  = soup.find_all(class_ = 'gall_date')      # 게시날짜 파싱
        게시물수 = len(links) - 20                       # 한 페이지에서 크롤링할 element 수
        
        # element 수만큼 parsing 결과 크롤링
        for i in range(게시물수):
            # 제목 크롤링
            dd = links[i].text
            if '\n' in dd:
                dd = dd.split('\n')[1]
            title_.append(dd)

            
            
            # 주소 크롤링
            url = 'https://gall.dcinside.com' + links[i].a.attrs['href']
            url_.append(url)
            
            
            
            # 조회수 크롤링
            views = count[i].text
            count_.append(views)
            
            
            
            # 날짜 크롤링
            date_time = date[i].text
            time_.append(date_time)
        num += 1
        driver.close()
        
        
        
        # 한 페이지의 게시물수가 20개 이하면 마지막 페이지로 간주하고 break
        if 게시물수 < 20: break
        # 한 페이지의 게시물수가 20개여도 마지막 페이지 일때가 있는데, 저번과 이번 parsing 이 같다면 break 
        if 게시물수 == 20:
            if title_.count(dd) > 1: break
    
    

    dcinside_info = pd.DataFrame({'제목': title_, '주소': url_, '조회수': count_, '게시날짜': time_ })
    dcinside_info = dcinside_info.drop(dcinside_info[dcinside_info['조회수'] == '-'].index)          # 운영자 글 제외
    dcinside_info = dcinside_info.astype({'조회수': 'int'})                                          # 자료형 변환
    dcinside_info = dcinside_info[dcinside_info['조회수'] >= 100]                                    # 조회수 100 이하 제외
    dcinside_info = dcinside_info[dcinside_info['게시날짜'] == day_back].reset_index(drop=True)     # 어제 등록된 글 아니면 제외
    
    
    
    # 게시물 내용까지 content 컬럼에 저장
    content_list = []
    for number in range(len(dcinside_info)):
        driver = webdriver.Chrome('chromedriver.exe')
        target_url = dcinside_info.loc[number,'주소']
        driver.get(target_url)
        time.sleep(sleeptime)
        
        content = driver.find_element_by_css_selector('div.write_div').text.replace('\n','')
        content_list.append(content)
        driver.close()
        
    dcinside_info['본문'] = content_list
    dcinside_info = dcinside_info.drop('게시날짜', axis=1)
    
    
    
    title_final   = []     # 댓글 모을 제목 리스트
    comment_final = []     # 제목별 댓글 리스트
    
    # 게시글 주소를 돌며 크롤링
    urls = list(dcinside_info['주소'].values)     # dcinside_info에서 주소만 추출
    for url in urls:
        # 드라이버 실행
        driver = webdriver.Chrome('chromedriver.exe')
        driver.get(url)
        time.sleep(sleeptime)
        
        
        
        soup = BeautifulSoup(driver.page_source, 'html.parser') 
        main_title   = soup.select('span.title_subject')[0].text     # 제목 파싱
        comment_list = soup.select('p.usertxt')                      # 댓글내용 파싱
        num = len(comment_list)                                      # 게시글당 댓글 수
        
        
        
        # 댓글내용 기본 전처리
        for k in range(num):
            title_final.append(main_title)
            
            temp_comment = comment_list[k].text
            temp_comment = temp_comment.replace('\n', '')
            temp_comment = temp_comment.replace('\t', '')
            temp_comment = temp_comment.replace('    ', '')
            comment_final.append(temp_comment)
        driver.close()
        
 

    dcinside_comment = pd.DataFrame({'제목': title_final, '댓글': comment_final})
    return dcinside_info, dcinside_comment

### 크롤링 예시

In [4]:
dcinside_두산중공업_info, dcinside_두산중공업_comment = get_dcinside_info('두산중공업')

In [5]:
dcinside_두산중공업_info

,제목,주소,조회수,본문
0,뭘 사야 할지 모르겠다는 사람들을 위한 글,https://gall.dcinside.com/board/view/?id=neost...,1869,"영화 아이언맨을 봤냐?토니스타크는 얼떨결에 입는로봇을 개발했고,그 덕에 히어로 놀이..."
1,요즘 연기금이 매수하는거,https://gall.dcinside.com/board/view/?id=neost...,284,두산중공업 <- 최근 20거래일 많이사는데 숏커버임?


In [6]:
dcinside_두산중공업_comment

,제목,댓글
0,뭘 사야 할지 모르겠다는 사람들을 위한 글,?
1,뭘 사야 할지 모르겠다는 사람들을 위한 글,ㅋㅋㅋ - dc App
2,뭘 사야 할지 모르겠다는 사람들을 위한 글,웃겼어...ㅋㅋ
3,뭘 사야 할지 모르겠다는 사람들을 위한 글,멋지다 - dc App
4,뭘 사야 할지 모르겠다는 사람들을 위한 글,명작이다 - dc App
5,뭘 사야 할지 모르겠다는 사람들을 위한 글,ㅋㅋㅋㅋㅋㅋㅋㅋ
6,뭘 사야 할지 모르겠다는 사람들을 위한 글,??
7,뭘 사야 할지 모르겠다는 사람들을 위한 글,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ - dc App
8,뭘 사야 할지 모르겠다는 사람들을 위한 글,얼마에 쳐 물렸냐? 난 탈출각만 보고 있는데
9,뭘 사야 할지 모르겠다는 사람들을 위한 글,물리셨다고요? - dc App
